In [ ]:
!pip3 install optuna==0.14.0
!pip install fastparquet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 91 kB 4.2 MB/s 
     |████████████████████████████████| 81 kB 7.6 MB/s 
     |████████████████████████████████| 209 kB 66.8 MB/s 
     |████████████████████████████████| 78 kB 8.8 MB/s 
     |████████████████████████████████| 78 kB 8.3 MB/s 
     |████████████████████████████████| 50 kB 5.9 MB/s 
     |████████████████████████████████| 112 kB 87.0 MB/s 
     |████████████████████████████████| 147 kB 80.1 MB/s 
  Created wheel for optuna: filename=optuna-0.14.0-py3-none-any.whl size=125709 sha256=136b264809c54a3f78f6e68580a63829fd0bac33230742867ff8da9ad70b6ee5
  Stored in directory: /root/.cache/pip/wheels/c1/41/64/03b183676c5d5e978de160cab6268d5b4fb095dff63f720e01
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=ea5abc6289b04e04b8baabb7de877115f27cbbd0061b85e5f0315c485fcbfbe3
  Stored in directory: /root/.ca

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 5.1 MB/s 
     |████████████████████████████████| 1.6 MB 82.4 MB/s 


In [ ]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf

from keras.datasets import mnist
from keras.models import Sequential,Model
from keras.layers import Activation, Flatten, Dense, Conv2D, Conv1D,Input
from keras.layers import MaxPooling1D, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD, Adagrad, RMSprop, Adam, Adadelta, Adamax, Nadam
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from keras.layers.core import *
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
import keras.backend as K
import optuna
import math
import random
from copy import deepcopy
from sklearn.utils import shuffle

from scipy.stats import sem
from numpy import mean
from numpy import std
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score

In [ ]:
print('Keras', keras.__version__)
print('TensorFlow', tf.__version__)
# import Optuna and OptKeras after Keras
print('Optuna', optuna.__version__)

Keras 2.9.0
TensorFlow 2.9.2
Optuna 0.14.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_df = pd.read_parquet("/content/drive/MyDrive/New Data/11features/100K11F_train_main.parquet.snappy",engine='fastparquet')
val_df = pd.read_parquet("/content/drive/MyDrive/New Data/11features/100K11F_val_main.parquet.snappy",engine='fastparquet')
test_df = pd.read_parquet("/content/drive/MyDrive/New Data/11features/100K11F_test_main.parquet.snappy",engine='fastparquet')

In [ ]:
train_df

,r_id,r_stars,r_stars_square,r_length,u_friends_count,u_review_count,u_month_age,b_stars,b_review_count,r_sen,r_sub,r_rea,r_useful
0,2486117,5,25,41,1,8,35.657224,1.5,13,0.099375,0.460000,74.49,2
1,3238107,5,25,64,174,247,1.024218,3.5,77,0.300000,0.260000,75.61,2
2,5581800,5,25,239,525,603,125.863690,4.0,1646,0.202032,0.444942,84.17,3
3,5953680,4,16,129,130,183,129.975625,4.5,356,0.129097,0.444861,83.76,4
4,2208283,1,1,537,223,1,0.000111,3.5,375,0.014266,0.300989,76.86,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,2065766,2,4,61,165,108,0.640257,2.5,9,-0.008333,0.354167,86.91,2
79996,4640630,4,16,74,78,467,47.038862,2.5,40,0.238016,0.480476,84.37,2
79997,4218981,5,25,98,12,88,79.607721,4.0,30,0.123629,0.247150,74.49,2
79998,4633311,4,16,656,173,206,41.955031,2.5,485,0.069350,0.415728,82.54,6


In [ ]:
training_withaim=train_df.drop(labels="r_id", axis=1)
valing_withaim=val_df.drop(labels="r_id",axis=1)
testing_withaim=test_df.drop(labels="r_id", axis=1)

#Check the NaN in data and drop them
imp_train=SimpleImputer(missing_values=np.NaN)
training=pd.DataFrame(imp_train.fit_transform(training_withaim))

imp_test=SimpleImputer(missing_values=np.NaN)
testing=pd.DataFrame(imp_test.fit_transform(testing_withaim))

imp_val=SimpleImputer(missing_values=np.NaN)
valing=pd.DataFrame(imp_val.fit_transform(valing_withaim))

# There aren't any nan data in the dataframe 
training = training.iloc[: , 0:11]
testing = testing.iloc[: , 0:11]
valing=valing.iloc[:,0:11]
scaler = StandardScaler()

# scale skewed features
std_train_df = train_df.copy(deep=True)
std_train_df = scaler.fit_transform(training)
std_test_df = test_df.copy(deep=True)
std_test_df = scaler.transform(testing)
std_val_df = val_df.copy(deep=True)
std_val_df = scaler.transform(valing)
#std_test_df[['r_stars','r_stars_square','r_length', 'u_friends_count', 'u_review_count', 'u_month_age', 'b_stars','b_review_count','r_rea']] = scaler.transform(test_df[['r_stars','r_stars_square','r_length', 'u_friends_count', 'u_review_count', 'u_month_age', 'b_stars','b_review_count','r_rea']])

std_train_df = pd.DataFrame(std_train_df)
std_val_df = pd.DataFrame(std_val_df)
std_test_df = pd.DataFrame(std_test_df)

training = std_train_df.iloc[: , 0:12]
valing = std_val_df.iloc[: , 0:12]
testing = std_test_df.iloc[: , 0:12]

labelsForTrain=training_withaim.iloc[: , -1]
labelsForVal=valing_withaim.iloc[: , -1]
labelsForTest=testing_withaim.iloc[: , -1]
input_shape = training.shape

In [ ]:
labelsForTrain

0        2
1        2
2        3
3        4
4        6
        ..
79995    2
79996    2
79997    2
79998    6
79999    2
Name: r_useful, Length: 80000, dtype: int64

In [ ]:
training

,0,1,2,3,4,5,6,7,8,9,10
0,0.902431,0.996586,-0.805509,-0.329454,-0.380837,-0.205818,-2.842089,-0.481997,-0.428646,0.106826,-0.509433
1,0.902431,0.996586,-0.603075,-0.026023,0.120685,-1.203206,-0.314664,-0.387357,0.611427,-1.250932,-0.406493
2,0.902431,0.996586,0.937184,0.589607,0.867722,2.392019,0.317192,1.932788,0.103544,0.004600,0.380261
3,0.253649,0.062198,-0.030979,-0.103196,-0.013614,2.510438,0.949048,0.025212,-0.274562,0.004050,0.342578
4,-1.692696,-1.495115,3.560026,0.059919,-0.395526,-1.232699,-0.314664,0.053308,-0.869865,-0.972666,-0.291605
...,...,...,...,...,...,...,...,...,...,...,...
79995,-1.043914,-1.183653,-0.629480,-0.041809,-0.170995,-1.214264,-1.578376,-0.487912,-0.987022,-0.611652,0.632096
79996,0.253649,0.062198,-0.515060,-0.194401,0.582337,0.121960,-1.578376,-0.442071,0.290091,0.245833,0.398643
79997,0.902431,0.996586,-0.303825,-0.310161,-0.212964,1.059903,0.317192,-0.456858,-0.302909,-1.338167,-0.509433
79998,0.253649,0.062198,4.607402,-0.027777,0.034650,-0.024449,-1.578376,0.215969,-0.584301,-0.193727,0.230447


In [ ]:
def create_model(activation, num_hidden_layer, num_hidden_unit):
  inputs = Input(shape=(training.shape[1],))
  model = inputs
  for i in range(1,num_hidden_layer):
    model = Dense(num_hidden_unit, activation=activation,)(model)
        
        
  model = Dense(1,)(model)
  model = Model(inputs, model)

  return model

In [ ]:
kfold = KFold(n_splits=5, shuffle=True)
def objective(trial:optuna.Trial,data_train,result_train):
  K.clear_session()
    
  activation = trial.suggest_categorical('activation',['relu','tanh','linear'])
  #Leave fewer optimizer
    
  num_hidden_layer = trial.suggest_int('num_hidden_layer',2,4)
  #get more features per layer, add num of hidden unit if have time

  #define the number of unit with 2^n
  i = trial.suggest_int('i',6,10)
  num_hidden_unit = 2**i
  
  #Try to adjust learning_rate

  learning_rate = trial.suggest_loguniform('learning_rate', 0.0001,0.01)
  
  # Gradient Clipping

  optimizer = Adam(learning_rate=learning_rate,clipnorm=1.0)
    
  num_folds = 3

  loss_per_fold = []
  es = EarlyStopping(monitor='val_mse', patience=5)

  model = create_model(activation,num_hidden_layer,num_hidden_unit)
  model_list.append(model)
  model.compile(loss='mse',optimizer=optimizer,metrics=['mse','mae'])
  #learning scheduler
  reduce_lr = ReduceLROnPlateau(monitor='val_mse', factor=0.2,
                              patience=5, min_lr=0.001)
  fold_no=1
      # Fit data to model
  score_int=[]

  for train,test in kfold.split(data_train,result_train):

    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    new_train_CV=data_train.iloc[train]
    new_label=result_train.iloc[train]

    val_CV=data_train.iloc[test]
    val_result=result_train.iloc[test]

    # Fit data to model
    history = model.fit(new_train_CV,new_label,
                batch_size=64,
                epochs=100,
                verbose=2,
                validation_data=(val_CV,val_result),
                validation_batch_size=64,
                callbacks=[es,reduce_lr])
    
    scores=model.evaluate(val_CV,val_result,verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}')
    loss_per_fold.append(scores[0])
    # Increase fold number
    score_int.append(round(scores[1],2))
    fold_no = fold_no + 1  
    
  mse = np.array(history.history['mse'])

  #loss_per_fold.append(scores[0])

  history_list.append(history)
  return np.mean(score_int)

In [ ]:
model_list=[]
history_list=[]
study_name = 'NN_study'
study = optuna.create_study(study_name=study_name, load_if_exists=True)
func = lambda trial: objective(trial,training,labelsForTrain)
study.optimize(func, n_trials=50,)

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100
1000/1000 - 3s - loss: 16.2763 - mse: 16.2763 - mae: 1.6251 - val_loss: 12.0089 - val_mse: 12.0089 - val_mae: 1.6946 - lr: 0.0019 - 3s/epoch - 3ms/step
Epoch 2/100
1000/1000 - 2s - loss: 15.9967 - mse: 15.9967 - mae: 1.6034 - val_loss: 11.9670 - val_mse: 11.9670 - val_mae: 1.5578 - lr: 0.0019 - 2s/epoch - 2ms/step
Epoch 3/100
1000/1000 - 2s - loss: 15.9137 - mse: 15.9137 - mae: 1.6024 - val_loss: 11.8501 - val_mse: 11.8501 - val_mae: 1.5811 - lr: 0.0019 - 2s/epoch - 2ms/step
Epoch 4/100
1000/1000 - 2s - loss: 15.8924 - mse: 15.8924 - mae: 1.5940 - val_loss: 11.8098 - val_mse: 11.8098 - val_mae: 1.6091 - lr: 0.0019 - 2s/epoch - 2ms/step
Epoch 5/100
1000/1000 - 2s - loss: 15.7952 - mse: 15.7952 - mae: 1.5942 - val_loss: 11.7593 - val_mse: 11.7593 - val_mae: 1.6272 - lr: 0.0019 - 2s/epoch - 2ms/step
Epoch 6/100
1000/1000 - 2s - loss: 15.7921 - mse: 15.7921 - mae: 1.5897 - val_loss:

[I 2022-10-14 11:11:54,696] Finished trial#0 resulted in value: 14.751999999999999. Current best value is 14.751999999999999 with parameters: {'activation': 'relu', 'num_hidden_layer': 3, 'i': 7, 'learning_rate': 0.0018797852452371259}.


Score for fold 5: loss of 11.857675552368164
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100
1000/1000 - 2s - loss: 20.7118 - mse: 20.7118 - mae: 2.0586 - val_loss: 16.6573 - val_mse: 16.6573 - val_mae: 1.5344 - lr: 1.3816e-04 - 2s/epoch - 2ms/step
Epoch 2/100
1000/1000 - 2s - loss: 16.0298 - mse: 16.0298 - mae: 1.5819 - val_loss: 15.4783 - val_mse: 15.4783 - val_mae: 1.6289 - lr: 1.3816e-04 - 2s/epoch - 2ms/step
Epoch 3/100
1000/1000 - 2s - loss: 15.7339 - mse: 15.7339 - mae: 1.6238 - val_loss: 15.3959 - val_mse: 15.3959 - val_mae: 1.6246 - lr: 1.3816e-04 - 2s/epoch - 2ms/step
Epoch 4/100
1000/1000 - 2s - loss: 15.6711 - mse: 15.6711 - mae: 1.6212 - val_loss: 15.3487 - val_mse: 15.3487 - val_mae: 1.6163 - lr: 1.3816e-04 - 2s/epoch - 2ms/step
Epoch 5/100
1000/1000 - 2s - loss: 15.6302 - mse: 15.6302 - mae: 1.6195 - val_loss: 15.3098 - val_mse: 15.3098 - val_mae: 1.6184 - lr: 1.3816e-04 - 2s/epoch - 2ms/step
Epoch 6/100
1000/1

[I 2022-10-14 11:14:42,777] Finished trial#1 resulted in value: 15.076000000000002. Current best value is 14.751999999999999 with parameters: {'activation': 'relu', 'num_hidden_layer': 3, 'i': 7, 'learning_rate': 0.0018797852452371259}.


Score for fold 5: loss of 9.731948852539062
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100
1000/1000 - 3s - loss: 16.6013 - mse: 16.6013 - mae: 1.6632 - val_loss: 12.7807 - val_mse: 12.7807 - val_mae: 1.6318 - lr: 0.0080 - 3s/epoch - 3ms/step
Epoch 2/100
1000/1000 - 2s - loss: 16.2661 - mse: 16.2661 - mae: 1.6331 - val_loss: 12.6939 - val_mse: 12.6939 - val_mae: 1.5776 - lr: 0.0080 - 2s/epoch - 2ms/step
Epoch 3/100
1000/1000 - 2s - loss: 16.1153 - mse: 16.1153 - mae: 1.6307 - val_loss: 12.8158 - val_mse: 12.8158 - val_mae: 1.5792 - lr: 0.0080 - 2s/epoch - 2ms/step
Epoch 4/100
1000/1000 - 2s - loss: 16.0705 - mse: 16.0705 - mae: 1.6348 - val_loss: 12.7401 - val_mse: 12.7401 - val_mae: 1.5930 - lr: 0.0080 - 2s/epoch - 2ms/step
Epoch 5/100
1000/1000 - 2s - loss: 16.0351 - mse: 16.0351 - mae: 1.6264 - val_loss: 12.5556 - val_mse: 12.5556 - val_mae: 1.6409 - lr: 0.0080 - 2s/epoch - 2ms/step
Epoch 6/100
1000/1000 - 2s - loss: 16.0

...

Score for fold 5: loss of 24.161746978759766
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100
1000/1000 - 6s - loss: 12.5974 - mse: 12.5974 - mae: 1.6039 - val_loss: 26.5185 - val_mse: 26.5185 - val_mae: 1.7608 - lr: 3.5237e-04 - 6s/epoch - 6ms/step
Epoch 2/100
1000/1000 - 5s - loss: 12.2798 - mse: 12.2798 - mae: 1.5870 - val_loss: 26.4772 - val_mse: 26.4772 - val_mae: 1.7252 - lr: 3.5237e-04 - 5s/epoch - 5ms/step
Epoch 3/100
1000/1000 - 5s - loss: 12.1792 - mse: 12.1792 - mae: 1.5814 - val_loss: 26.5550 - val_mse: 26.5550 - val_mae: 1.6256 - lr: 3.5237e-04 - 5s/epoch - 5ms/step
Epoch 4/100
1000/1000 - 5s - loss: 12.1734 - mse: 12.1734 - mae: 1.5827 - val_loss: 26.6399 - val_mse: 26.6399 - val_mae: 1.6935 - lr: 3.5237e-04 - 5s/epoch - 5ms/step
Epoch 5/100
1000/1000 - 5s - loss: 12.0885 - mse: 12.0885 - mae: 1.5730 - val_loss: 26.4851 - val_mse: 26.4851 - val_mae: 1.6398 - lr: 3.5237e-04 - 5s/epoch - 5ms/step
Epoch 6/100
1000/1

[I 2022-10-14 14:26:33,060] Finished trial#48 resulted in value: 14.955999999999998. Current best value is 14.266 with parameters: {'activation': 'relu', 'num_hidden_layer': 4, 'i': 10, 'learning_rate': 0.00010332846452133283}.


Score for fold 5: loss of 8.405388832092285
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100
1000/1000 - 4s - loss: 16.6705 - mse: 16.6705 - mae: 1.6170 - val_loss: 10.6649 - val_mse: 10.6649 - val_mae: 1.5443 - lr: 5.0769e-04 - 4s/epoch - 4ms/step
Epoch 2/100
1000/1000 - 4s - loss: 16.2626 - mse: 16.2626 - mae: 1.5970 - val_loss: 10.5537 - val_mse: 10.5537 - val_mae: 1.6244 - lr: 5.0769e-04 - 4s/epoch - 4ms/step
Epoch 3/100
1000/1000 - 4s - loss: 16.2118 - mse: 16.2118 - mae: 1.5943 - val_loss: 10.6541 - val_mse: 10.6541 - val_mae: 1.5515 - lr: 5.0769e-04 - 4s/epoch - 4ms/step
Epoch 4/100
1000/1000 - 4s - loss: 16.1474 - mse: 16.1474 - mae: 1.5892 - val_loss: 10.4382 - val_mse: 10.4382 - val_mae: 1.5828 - lr: 5.0769e-04 - 4s/epoch - 4ms/step
Epoch 5/100
1000/1000 - 4s - loss: 16.1347 - mse: 16.1347 - mae: 1.5873 - val_loss: 10.4356 - val_mse: 10.4356 - val_mae: 1.5418 - lr: 5.0769e-04 - 4s/epoch - 4ms/step
Epoch 6/100
1000/10

[I 2022-10-14 14:29:25,145] Finished trial#49 resulted in value: 14.790000000000001. Current best value is 14.266 with parameters: {'activation': 'relu', 'num_hidden_layer': 4, 'i': 10, 'learning_rate': 0.00010332846452133283}.


Score for fold 5: loss of 9.220843315124512


In [ ]:
#{'activation': 'tanh', 'num_hidden_layer': 4, 'i': 9, 'learning_rate': 0.00039524792654887494}.
optimizer = Adam(learning_rate=0.00010332846452133283 ,clipnorm=1.0)
model_1 = create_model(activation="tanh",num_hidden_layer=4,num_hidden_unit=512)

es = EarlyStopping(monitor='val_mse', patience=10)
reduce_lr = ReduceLROnPlateau(monitor='val_mse', factor=0.2,
                              patience=20, min_lr=0.001)
model_1.compile(loss='mse',optimizer=optimizer,metrics=['mse','mae'])
history = model_1.fit(training,labelsForTrain,
                batch_size=64,
                epochs=500,
                verbose=2,
                validation_data=(valing,labelsForVal),
                validation_batch_size=64,
                callbacks=[es,reduce_lr])

Epoch 1/500
1250/1250 - 6s - loss: 15.9909 - mse: 15.9909 - mae: 1.6802 - val_loss: 13.9783 - val_mse: 13.9783 - val_mae: 1.6952 - lr: 1.0333e-04 - 6s/epoch - 5ms/step
Epoch 2/500
1250/1250 - 5s - loss: 15.4380 - mse: 15.4380 - mae: 1.6140 - val_loss: 13.7686 - val_mse: 13.7686 - val_mae: 1.6166 - lr: 1.0333e-04 - 5s/epoch - 4ms/step
Epoch 3/500
1250/1250 - 5s - loss: 15.3375 - mse: 15.3375 - mae: 1.6038 - val_loss: 13.5927 - val_mse: 13.5927 - val_mae: 1.7006 - lr: 1.0333e-04 - 5s/epoch - 4ms/step
Epoch 4/500
1250/1250 - 5s - loss: 15.2647 - mse: 15.2647 - mae: 1.6013 - val_loss: 13.6895 - val_mse: 13.6895 - val_mae: 1.7494 - lr: 1.0333e-04 - 5s/epoch - 4ms/step
Epoch 5/500
1250/1250 - 5s - loss: 15.2675 - mse: 15.2675 - mae: 1.5985 - val_loss: 13.5877 - val_mse: 13.5877 - val_mae: 1.5684 - lr: 1.0333e-04 - 5s/epoch - 4ms/step
Epoch 6/500
1250/1250 - 5s - loss: 15.2242 - mse: 15.2242 - mae: 1.5951 - val_loss: 13.5446 - val_mse: 13.5446 - val_mae: 1.6720 - lr: 1.0333e-04 - 5s/epoch - 4

In [ ]:
results_model1 = model_1.evaluate(testing, labelsForTest, batch_size=32)

313/313 [==============================] - 1s 2ms/step - loss: 11.6705 - mse: 11.6705 - mae: 1.5727


## Shuffle & Repetation 2

In [ ]:
# Shuffle the data after train
shuffled1 = shuffle(train_df, random_state=1)

In [ ]:
def process_shuffle_dataset(dataset):
  training_withaim=dataset.drop(labels="r_id", axis=1)
  imp_train=SimpleImputer(missing_values=np.NaN)
  training_shuffled=pd.DataFrame(imp_train.fit_transform(training_withaim))
  training_shuffled = training_shuffled.iloc[: , 0:11]
  scaler = StandardScaler()
  std_train_df = dataset.copy(deep=True)
  std_train_df = scaler.fit_transform(training_shuffled)
  std_train_df = pd.DataFrame(std_train_df)
  training_shuffled = std_train_df.iloc[: , 0:12]
  labelsForTrain_shuffled=training_withaim.iloc[: , -1]
  return(training_shuffled,labelsForTrain_shuffled)

In [ ]:
training_shuffled1,labelsForTrain_shuffled1=process_shuffle_dataset(shuffled1)

In [ ]:
model_list=[]
history_list=[]
study_name = 'NN_study_2'
study = optuna.create_study(study_name=study_name, load_if_exists=True)
func = lambda trial: objective(trial,training_shuffled1,labelsForTrain_shuffled1)
study.optimize(func, n_trials=50,)

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100
1000/1000 - 3s - loss: 15.0282 - mse: 15.0282 - mae: 1.6655 - val_loss: 18.7563 - val_mse: 18.7563 - val_mae: 1.5745 - lr: 6.1633e-04 - 3s/epoch - 3ms/step
Epoch 2/100
1000/1000 - 2s - loss: 14.5845 - mse: 14.5845 - mae: 1.6152 - val_loss: 18.6136 - val_mse: 18.6136 - val_mae: 1.5898 - lr: 6.1633e-04 - 2s/epoch - 2ms/step
Epoch 3/100
1000/1000 - 2s - loss: 14.4450 - mse: 14.4450 - mae: 1.6062 - val_loss: 18.4968 - val_mse: 18.4968 - val_mae: 1.6463 - lr: 6.1633e-04 - 2s/epoch - 2ms/step
Epoch 4/100
1000/1000 - 2s - loss: 14.4048 - mse: 14.4048 - mae: 1.6021 - val_loss: 18.4354 - val_mse: 18.4354 - val_mae: 1.5850 - lr: 6.1633e-04 - 2s/epoch - 2ms/step
Epoch 5/100
1000/1000 - 2s - loss: 14.3877 - mse: 14.3877 - mae: 1.6041 - val_loss: 18.4718 - val_mse: 18.4718 - val_mae: 1.6301 - lr: 6.1633e-04 - 2s/epoch - 2ms/step
Epoch 6/100
1000/1000 - 2s - loss: 14.3005 - mse: 14.3005 - mae

[I 2022-10-14 14:31:41,781] Finished trial#0 resulted in value: 14.828. Current best value is 14.828 with parameters: {'activation': 'tanh', 'num_hidden_layer': 4, 'i': 7, 'learning_rate': 0.0006163259286815976}.


------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100
1000/1000 - 3s - loss: 18.0802 - mse: 18.0802 - mae: 1.7739 - val_loss: 12.9341 - val_mse: 12.9341 - val_mae: 1.6154 - lr: 1.7552e-04 - 3s/epoch - 3ms/step
Epoch 2/100
1000/1000 - 2s - loss: 16.1743 - mse: 16.1743 - mae: 1.6275 - val_loss: 12.9060 - val_mse: 12.9060 - val_mae: 1.6170 - lr: 1.7552e-04 - 2s/epoch - 2ms/step
Epoch 3/100
1000/1000 - 2s - loss: 16.1437 - mse: 16.1437 - mae: 1.6236 - val_loss: 12.8711 - val_mse: 12.8711 - val_mae: 1.6199 - lr: 1.7552e-04 - 2s/epoch - 2ms/step
Epoch 4/100
1000/1000 - 2s - loss: 16.1127 - mse: 16.1127 - mae: 1.6268 - val_loss: 12.8848 - val_mse: 12.8848 - val_mae: 1.5719 - lr: 1.7552e-04 - 2s/epoch - 2ms/step
Epoch 5/100
1000/1000 - 2s - loss: 16.0885 - mse: 16.0885 - mae: 1.6186 - val_loss: 12.8518 - val_mse: 12.8518 - val_mae: 1.5754 - lr: 1.7552e-04 - 2s/epoch - 2ms/step
Epoch 6/100
1000/1000 - 2s - loss: 16.0632 - mse: 16.0632 - mae

[I 2022-10-14 14:33:52,501] Finished trial#1 resulted in value: 15.104. Current best value is 14.828 with parameters: {'activation': 'tanh', 'num_hidden_layer': 4, 'i': 7, 'learning_rate': 0.0006163259286815976}.


Score for fold 5: loss of 15.041754722595215
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100
1000/1000 - 3s - loss: 17.5532 - mse: 17.5532 - mae: 1.7328 - val_loss: 11.9741 - val_mse: 11.9741 - val_mae: 1.6121 - lr: 2.6768e-04 - 3s/epoch - 3ms/step
Epoch 2/100
1000/1000 - 2s - loss: 16.5715 - mse: 16.5715 - mae: 1.6271 - val_loss: 11.9287 - val_mse: 11.9287 - val_mae: 1.6335 - lr: 2.6768e-04 - 2s/epoch - 2ms/step
Epoch 3/100
1000/1000 - 2s - loss: 16.6053 - mse: 16.6053 - mae: 1.6280 - val_loss: 11.8918 - val_mse: 11.8918 - val_mae: 1.6397 - lr: 2.6768e-04 - 2s/epoch - 2ms/step
Epoch 4/100
1000/1000 - 2s - loss: 16.5827 - mse: 16.5827 - mae: 1.6283 - val_loss: 11.9450 - val_mse: 11.9450 - val_mae: 1.6245 - lr: 2.6768e-04 - 2s/epoch - 2ms/step
Epoch 5/100
1000/1000 - 2s - loss: 16.5908 - mse: 16.5908 - mae: 1.6236 - val_loss: 11.9075 - val_mse: 11.9075 - val_mae: 1.6365 - lr: 2.6768e-04 - 2s/epoch - 2ms/step
Epoch 6/100
1000/1

...

Score for fold 5: loss of 16.107099533081055
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100
1000/1000 - 3s - loss: 15.9576 - mse: 15.9576 - mae: 1.6076 - val_loss: 13.8855 - val_mse: 13.8855 - val_mae: 1.6954 - lr: 0.0010 - 3s/epoch - 3ms/step
Epoch 2/100
1000/1000 - 2s - loss: 15.5370 - mse: 15.5370 - mae: 1.5900 - val_loss: 13.7742 - val_mse: 13.7742 - val_mae: 1.6663 - lr: 0.0010 - 2s/epoch - 2ms/step
Epoch 3/100
1000/1000 - 2s - loss: 15.4511 - mse: 15.4511 - mae: 1.5853 - val_loss: 13.7006 - val_mse: 13.7006 - val_mae: 1.6232 - lr: 0.0010 - 2s/epoch - 2ms/step
Epoch 4/100
1000/1000 - 2s - loss: 15.3958 - mse: 15.3958 - mae: 1.5809 - val_loss: 13.7239 - val_mse: 13.7239 - val_mae: 1.6140 - lr: 0.0010 - 2s/epoch - 2ms/step
Epoch 5/100
1000/1000 - 2s - loss: 15.3697 - mse: 15.3697 - mae: 1.5752 - val_loss: 13.6284 - val_mse: 13.6284 - val_mae: 1.6228 - lr: 0.0010 - 2s/epoch - 2ms/step
Epoch 6/100
1000/1000 - 2s - loss: 15.

[I 2022-10-14 16:36:02,205] Finished trial#48 resulted in value: 14.876000000000001. Current best value is 14.594 with parameters: {'activation': 'relu', 'num_hidden_layer': 4, 'i': 6, 'learning_rate': 0.000577682381030866}.


Score for fold 5: loss of 11.7587890625
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100
1000/1000 - 3s - loss: 14.9147 - mse: 14.9147 - mae: 1.6392 - val_loss: 17.7069 - val_mse: 17.7069 - val_mae: 1.5280 - lr: 0.0021 - 3s/epoch - 3ms/step
Epoch 2/100
1000/1000 - 2s - loss: 14.6441 - mse: 14.6441 - mae: 1.6208 - val_loss: 17.4358 - val_mse: 17.4358 - val_mae: 1.5864 - lr: 0.0021 - 2s/epoch - 2ms/step
Epoch 3/100
1000/1000 - 3s - loss: 14.5709 - mse: 14.5709 - mae: 1.6149 - val_loss: 17.5000 - val_mse: 17.5000 - val_mae: 1.5801 - lr: 0.0021 - 3s/epoch - 3ms/step
Epoch 4/100
1000/1000 - 3s - loss: 14.5100 - mse: 14.5100 - mae: 1.6039 - val_loss: 17.5669 - val_mse: 17.5669 - val_mae: 1.6655 - lr: 0.0021 - 3s/epoch - 3ms/step
Epoch 5/100
1000/1000 - 3s - loss: 14.4332 - mse: 14.4332 - mae: 1.6066 - val_loss: 17.3003 - val_mse: 17.3003 - val_mae: 1.5059 - lr: 0.0021 - 3s/epoch - 3ms/step
Epoch 6/100
1000/1000 - 3s - loss: 14.4249 

[I 2022-10-14 16:37:48,691] Finished trial#49 resulted in value: 15.051999999999998. Current best value is 14.594 with parameters: {'activation': 'relu', 'num_hidden_layer': 4, 'i': 6, 'learning_rate': 0.000577682381030866}.


Score for fold 5: loss of 12.82724380493164


In [ ]:
#{'activation': 'relu', 'num_hidden_layer': 4, 'i': 6, 'learning_rate': 0.000577682381030866}
optimizer = Adam(learning_rate=0.000577682381030866 ,clipnorm=1.0)
model_2 = create_model(activation="relu",num_hidden_layer=4,num_hidden_unit=64)

es = EarlyStopping(monitor='mse', patience=5)
reduce_lr = ReduceLROnPlateau(monitor='val_mse', factor=0.2,
                              patience=20, min_lr=0.001)
model_2.compile(loss='mse',optimizer=optimizer,metrics=['mse','mae'])
history = model_2.fit(training,labelsForTrain,batch_size=64,epochs=20,verbose=2,validation_data=(valing,labelsForVal),validation_batch_size=64,callbacks=[es,reduce_lr])

Epoch 1/20
1250/1250 - 3s - loss: 15.6789 - mse: 15.6789 - mae: 1.6297 - val_loss: 13.6266 - val_mse: 13.6266 - val_mae: 1.5842 - lr: 5.7768e-04 - 3s/epoch - 2ms/step
Epoch 2/20
1250/1250 - 2s - loss: 15.1218 - mse: 15.1218 - mae: 1.5981 - val_loss: 13.7104 - val_mse: 13.7104 - val_mae: 1.5945 - lr: 5.7768e-04 - 2s/epoch - 2ms/step
Epoch 3/20
1250/1250 - 2s - loss: 15.0900 - mse: 15.0900 - mae: 1.5918 - val_loss: 13.3743 - val_mse: 13.3743 - val_mae: 1.6734 - lr: 5.7768e-04 - 2s/epoch - 2ms/step
Epoch 4/20
1250/1250 - 2s - loss: 15.0565 - mse: 15.0565 - mae: 1.5889 - val_loss: 13.4266 - val_mse: 13.4266 - val_mae: 1.5840 - lr: 5.7768e-04 - 2s/epoch - 2ms/step
Epoch 5/20
1250/1250 - 2s - loss: 14.9781 - mse: 14.9781 - mae: 1.5840 - val_loss: 13.3230 - val_mse: 13.3230 - val_mae: 1.5776 - lr: 5.7768e-04 - 2s/epoch - 2ms/step
Epoch 6/20
1250/1250 - 2s - loss: 14.9715 - mse: 14.9715 - mae: 1.5828 - val_loss: 13.4451 - val_mse: 13.4451 - val_mae: 1.5467 - lr: 5.7768e-04 - 2s/epoch - 2ms/ste

In [ ]:
results_model2 = model_2.evaluate(testing, labelsForTest, batch_size=32)

313/313 [==============================] - 0s 1ms/step - loss: 11.8047 - mse: 11.8047 - mae: 1.5521


## Shuffle Repetation 3

In [ ]:
# Shuffle the data after train
shuffled2 = shuffle(train_df, random_state=2)

In [ ]:
training_shuffled2,labelsForTrain_shuffled2=process_shuffle_dataset(shuffled2)

In [ ]:
model_list=[]
history_list=[]
study_name = 'NN_study_3'
study = optuna.create_study(study_name=study_name, load_if_exists=True)
func = lambda trial: objective(trial,training_shuffled2,labelsForTrain_shuffled2) 
study.optimize(func, n_trials=50,)

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100
1000/1000 - 3s - loss: 16.2146 - mse: 16.2146 - mae: 1.6847 - val_loss: 15.7815 - val_mse: 15.7815 - val_mae: 1.6218 - lr: 3.4778e-04 - 3s/epoch - 3ms/step
Epoch 2/100
1000/1000 - 2s - loss: 15.4812 - mse: 15.4812 - mae: 1.6125 - val_loss: 15.6420 - val_mse: 15.6420 - val_mae: 1.6365 - lr: 3.4778e-04 - 2s/epoch - 2ms/step
Epoch 3/100
1000/1000 - 2s - loss: 15.3792 - mse: 15.3792 - mae: 1.6073 - val_loss: 15.4634 - val_mse: 15.4634 - val_mae: 1.5639 - lr: 3.4778e-04 - 2s/epoch - 2ms/step
Epoch 4/100
1000/1000 - 2s - loss: 15.2884 - mse: 15.2884 - mae: 1.6018 - val_loss: 15.3333 - val_mse: 15.3333 - val_mae: 1.5757 - lr: 3.4778e-04 - 2s/epoch - 2ms/step
Epoch 5/100
1000/1000 - 2s - loss: 15.2079 - mse: 15.2079 - mae: 1.5985 - val_loss: 15.2733 - val_mse: 15.2733 - val_mae: 1.6433 - lr: 3.4778e-04 - 2s/epoch - 2ms/step
Epoch 6/100
1000/1000 - 2s - loss: 15.1718 - mse: 15.1718 - mae

[I 2022-10-14 20:32:34,883] Finished trial#0 resulted in value: 14.864. Current best value is 14.864 with parameters: {'activation': 'tanh', 'num_hidden_layer': 4, 'i': 6, 'learning_rate': 0.0003477839020306978}.


------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100
1000/1000 - 3s - loss: 16.5482 - mse: 16.5482 - mae: 1.6479 - val_loss: 12.1395 - val_mse: 12.1395 - val_mae: 1.5700 - lr: 4.9576e-04 - 3s/epoch - 3ms/step
Epoch 2/100
1000/1000 - 2s - loss: 15.9349 - mse: 15.9349 - mae: 1.5993 - val_loss: 12.0229 - val_mse: 12.0229 - val_mae: 1.5408 - lr: 4.9576e-04 - 2s/epoch - 2ms/step
Epoch 3/100
1000/1000 - 2s - loss: 15.8630 - mse: 15.8630 - mae: 1.5932 - val_loss: 11.9242 - val_mse: 11.9242 - val_mae: 1.6121 - lr: 4.9576e-04 - 2s/epoch - 2ms/step
Epoch 4/100
1000/1000 - 2s - loss: 15.8001 - mse: 15.8001 - mae: 1.5948 - val_loss: 11.9091 - val_mse: 11.9091 - val_mae: 1.5533 - lr: 4.9576e-04 - 2s/epoch - 2ms/step
Epoch 5/100
1000/1000 - 2s - loss: 15.7582 - mse: 15.7582 - mae: 1.5938 - val_loss: 11.8464 - val_mse: 11.8464 - val_mae: 1.5461 - lr: 4.9576e-04 - 2s/epoch - 2ms/step
Epoch 6/100
1000/1000 - 2s - loss: 15.7378 - mse: 15.7378 - mae

[I 2022-10-14 20:33:58,639] Finished trial#1 resulted in value: 15.11. Current best value is 14.864 with parameters: {'activation': 'tanh', 'num_hidden_layer': 4, 'i': 6, 'learning_rate': 0.0003477839020306978}.


Score for fold 5: loss of 13.465463638305664
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100
1000/1000 - 11s - loss: 14.6271 - mse: 14.6271 - mae: 1.7276 - val_loss: 23.3357 - val_mse: 23.3357 - val_mae: 2.1713 - lr: 0.0013 - 11s/epoch - 11ms/step
Epoch 2/100
1000/1000 - 10s - loss: 14.3222 - mse: 14.3222 - mae: 1.6913 - val_loss: 22.1833 - val_mse: 22.1833 - val_mae: 1.8103 - lr: 0.0013 - 10s/epoch - 10ms/step
Epoch 3/100
1000/1000 - 9s - loss: 14.2439 - mse: 14.2439 - mae: 1.7082 - val_loss: 22.5607 - val_mse: 22.5607 - val_mae: 1.5195 - lr: 0.0013 - 9s/epoch - 9ms/step
Epoch 4/100
1000/1000 - 9s - loss: 14.1643 - mse: 14.1643 - mae: 1.7178 - val_loss: 23.6399 - val_mse: 23.6399 - val_mae: 1.7459 - lr: 0.0013 - 9s/epoch - 9ms/step
Epoch 5/100
1000/1000 - 9s - loss: 14.0643 - mse: 14.0643 - mae: 1.7008 - val_loss: 21.9044 - val_mse: 21.9044 - val_mae: 1.7154 - lr: 0.0013 - 9s/epoch - 9ms/step
Epoch 6/100
1000/1000 - 9s - los

...

Score for fold 5: loss of 13.104900360107422
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100
1000/1000 - 3s - loss: 16.5320 - mse: 16.5320 - mae: 1.6129 - val_loss: 12.9108 - val_mse: 12.9108 - val_mae: 1.6737 - lr: 1.4991e-04 - 3s/epoch - 3ms/step
Epoch 2/100
1000/1000 - 2s - loss: 15.7824 - mse: 15.7824 - mae: 1.5708 - val_loss: 12.8251 - val_mse: 12.8251 - val_mae: 1.6030 - lr: 1.4991e-04 - 2s/epoch - 2ms/step
Epoch 3/100
1000/1000 - 2s - loss: 15.7011 - mse: 15.7011 - mae: 1.5676 - val_loss: 12.9226 - val_mse: 12.9226 - val_mae: 1.5806 - lr: 1.4991e-04 - 2s/epoch - 2ms/step
Epoch 4/100
1000/1000 - 2s - loss: 15.6451 - mse: 15.6451 - mae: 1.5698 - val_loss: 12.7380 - val_mse: 12.7380 - val_mae: 1.6407 - lr: 1.4991e-04 - 2s/epoch - 2ms/step
Epoch 5/100
1000/1000 - 2s - loss: 15.5741 - mse: 15.5741 - mae: 1.5661 - val_loss: 12.7180 - val_mse: 12.7180 - val_mae: 1.6405 - lr: 1.4991e-04 - 2s/epoch - 2ms/step
Epoch 6/100
1000/1

[I 2022-10-14 22:00:22,727] Finished trial#48 resulted in value: 14.863999999999999. Current best value is 14.724 with parameters: {'activation': 'relu', 'num_hidden_layer': 3, 'i': 6, 'learning_rate': 0.0023989159773794054}.


Score for fold 5: loss of 9.683448791503906
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100
1000/1000 - 2s - loss: 16.3926 - mse: 16.3926 - mae: 1.6314 - val_loss: 11.2192 - val_mse: 11.2192 - val_mae: 1.5592 - lr: 0.0027 - 2s/epoch - 2ms/step
Epoch 2/100
1000/1000 - 2s - loss: 16.1855 - mse: 16.1855 - mae: 1.6141 - val_loss: 11.3460 - val_mse: 11.3460 - val_mae: 1.5422 - lr: 0.0027 - 2s/epoch - 2ms/step
Epoch 3/100
1000/1000 - 2s - loss: 16.0401 - mse: 16.0401 - mae: 1.6087 - val_loss: 11.2456 - val_mse: 11.2456 - val_mae: 1.5657 - lr: 0.0027 - 2s/epoch - 2ms/step
Epoch 4/100
1000/1000 - 2s - loss: 16.0770 - mse: 16.0770 - mae: 1.6046 - val_loss: 11.1208 - val_mse: 11.1208 - val_mae: 1.5693 - lr: 0.0027 - 2s/epoch - 2ms/step
Epoch 5/100
1000/1000 - 2s - loss: 16.0197 - mse: 16.0197 - mae: 1.6002 - val_loss: 11.1580 - val_mse: 11.1580 - val_mae: 1.6108 - lr: 0.0027 - 2s/epoch - 2ms/step
Epoch 6/100
1000/1000 - 2s - loss: 16.0

[I 2022-10-14 22:01:44,761] Finished trial#49 resulted in value: 14.912. Current best value is 14.724 with parameters: {'activation': 'relu', 'num_hidden_layer': 3, 'i': 6, 'learning_rate': 0.0023989159773794054}.


Score for fold 5: loss of 16.399837493896484


In [ ]:
#{'activation': 'relu', 'num_hidden_layer': 3, 'i': 6, 'learning_rate': 0.0023989159773794054}.
optimizer = Adam(learning_rate=0.0023989159773794054 ,clipnorm=1.0)
model_3 = create_model(activation="relu",num_hidden_layer=3,num_hidden_unit=64)

es = EarlyStopping(monitor='mse', patience=20)
reduce_lr = ReduceLROnPlateau(monitor='val_mse', factor=0.2,
                              patience=20, min_lr=0.001)
model_3.compile(loss='mse',optimizer=optimizer,metrics=['mse','mae'])
history = model_3.fit(training,labelsForTrain,
                batch_size=64,
                epochs=100,
                verbose=2,
                validation_data=(valing,labelsForVal),
                validation_batch_size=64,
                callbacks=[es,reduce_lr])

Epoch 1/100
1250/1250 - 2s - loss: 15.3970 - mse: 15.3970 - mae: 1.6125 - val_loss: 13.7548 - val_mse: 13.7548 - val_mae: 1.5799 - lr: 0.0024 - 2s/epoch - 2ms/step
Epoch 2/100
1250/1250 - 2s - loss: 15.1364 - mse: 15.1364 - mae: 1.5975 - val_loss: 13.3447 - val_mse: 13.3447 - val_mae: 1.6615 - lr: 0.0024 - 2s/epoch - 1ms/step
Epoch 3/100
1250/1250 - 2s - loss: 15.1007 - mse: 15.1007 - mae: 1.5920 - val_loss: 13.3369 - val_mse: 13.3369 - val_mae: 1.5922 - lr: 0.0024 - 2s/epoch - 1ms/step
Epoch 4/100
1250/1250 - 2s - loss: 15.0337 - mse: 15.0337 - mae: 1.5879 - val_loss: 13.3207 - val_mse: 13.3207 - val_mae: 1.6769 - lr: 0.0024 - 2s/epoch - 1ms/step
Epoch 5/100
1250/1250 - 2s - loss: 15.0871 - mse: 15.0871 - mae: 1.5891 - val_loss: 13.2484 - val_mse: 13.2484 - val_mae: 1.6134 - lr: 0.0024 - 2s/epoch - 1ms/step
Epoch 6/100
1250/1250 - 2s - loss: 15.0044 - mse: 15.0044 - mae: 1.5944 - val_loss: 13.5017 - val_mse: 13.5017 - val_mae: 1.5716 - lr: 0.0024 - 2s/epoch - 1ms/step
Epoch 7/100
1250

In [ ]:
results_model3 = model_3.evaluate(testing, labelsForTest, batch_size=32)

313/313 [==============================] - 0s 1ms/step - loss: 11.7022 - mse: 11.7022 - mae: 1.5718


## Shuffle Repetation 4

In [ ]:
# Shuffle the data after train
shuffled3 = shuffle(train_df, random_state=3)
training_shuffled3,labelsForTrain_shuffled3=process_shuffle_dataset(shuffled3)

In [ ]:
model_list=[]
history_list=[]
study_name = 'NN_study_4'
study = optuna.create_study(study_name=study_name, load_if_exists=True)
func = lambda trial: objective(trial,training_shuffled3,labelsForTrain_shuffled3)
study.optimize(func, n_trials=50,)

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100
1000/1000 - 4s - loss: 15.2986 - mse: 15.2986 - mae: 1.6165 - val_loss: 16.2284 - val_mse: 16.2284 - val_mae: 1.5737 - lr: 0.0017 - 4s/epoch - 4ms/step
Epoch 2/100
1000/1000 - 3s - loss: 14.9629 - mse: 14.9629 - mae: 1.5914 - val_loss: 15.9311 - val_mse: 15.9311 - val_mae: 1.5997 - lr: 0.0017 - 3s/epoch - 3ms/step
Epoch 3/100
1000/1000 - 3s - loss: 14.9289 - mse: 14.9289 - mae: 1.5874 - val_loss: 15.8437 - val_mse: 15.8437 - val_mae: 1.6022 - lr: 0.0017 - 3s/epoch - 3ms/step
Epoch 4/100
1000/1000 - 3s - loss: 14.8404 - mse: 14.8404 - mae: 1.5810 - val_loss: 15.9394 - val_mse: 15.9394 - val_mae: 1.6320 - lr: 0.0017 - 3s/epoch - 3ms/step
Epoch 5/100
1000/1000 - 4s - loss: 14.9030 - mse: 14.9030 - mae: 1.5782 - val_loss: 15.8178 - val_mse: 15.8178 - val_mae: 1.5926 - lr: 0.0017 - 4s/epoch - 4ms/step
Epoch 6/100
1000/1000 - 3s - loss: 14.8280 - mse: 14.8280 - mae: 1.5772 - val_loss:

[I 2022-10-15 05:09:44,739] Finished trial#0 resulted in value: 14.972. Current best value is 14.972 with parameters: {'activation': 'relu', 'num_hidden_layer': 3, 'i': 8, 'learning_rate': 0.0017060481633639777}.


Score for fold 5: loss of 21.398807525634766
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100
1000/1000 - 3s - loss: 15.8214 - mse: 15.8214 - mae: 1.6932 - val_loss: 18.8939 - val_mse: 18.8939 - val_mae: 1.6622 - lr: 3.6064e-04 - 3s/epoch - 3ms/step
Epoch 2/100
1000/1000 - 2s - loss: 14.7938 - mse: 14.7938 - mae: 1.5995 - val_loss: 18.7106 - val_mse: 18.7106 - val_mae: 1.6247 - lr: 3.6064e-04 - 2s/epoch - 2ms/step
Epoch 3/100
1000/1000 - 2s - loss: 14.6917 - mse: 14.6917 - mae: 1.5935 - val_loss: 18.5636 - val_mse: 18.5636 - val_mae: 1.6342 - lr: 3.6064e-04 - 2s/epoch - 2ms/step
Epoch 4/100
1000/1000 - 2s - loss: 14.6237 - mse: 14.6237 - mae: 1.5893 - val_loss: 18.3851 - val_mse: 18.3851 - val_mae: 1.6750 - lr: 3.6064e-04 - 2s/epoch - 2ms/step
Epoch 5/100
1000/1000 - 2s - loss: 14.5631 - mse: 14.5631 - mae: 1.5856 - val_loss: 18.3360 - val_mse: 18.3360 - val_mae: 1.6488 - lr: 3.6064e-04 - 2s/epoch - 2ms/step
Epoch 6/100
1000/1

[I 2022-10-15 05:12:31,874] Finished trial#1 resulted in value: 14.9. Current best value is 14.9 with parameters: {'activation': 'tanh', 'num_hidden_layer': 3, 'i': 6, 'learning_rate': 0.0003606405459716391}.


Score for fold 5: loss of 15.947320938110352
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100
1000/1000 - 3s - loss: 16.9103 - mse: 16.9103 - mae: 1.6817 - val_loss: 11.7166 - val_mse: 11.7166 - val_mae: 1.5915 - lr: 0.0082 - 3s/epoch - 3ms/step
Epoch 2/100
1000/1000 - 2s - loss: 16.7630 - mse: 16.7630 - mae: 1.6669 - val_loss: 11.9591 - val_mse: 11.9591 - val_mae: 1.7922 - lr: 0.0082 - 2s/epoch - 2ms/step
Epoch 3/100
1000/1000 - 2s - loss: 16.6542 - mse: 16.6542 - mae: 1.6570 - val_loss: 11.6039 - val_mse: 11.6039 - val_mae: 1.5640 - lr: 0.0082 - 2s/epoch - 2ms/step
Epoch 4/100
1000/1000 - 2s - loss: 16.4803 - mse: 16.4803 - mae: 1.6386 - val_loss: 11.8657 - val_mse: 11.8657 - val_mae: 1.5701 - lr: 0.0082 - 2s/epoch - 2ms/step
Epoch 5/100
1000/1000 - 2s - loss: 16.3960 - mse: 16.3960 - mae: 1.6367 - val_loss: 11.9266 - val_mse: 11.9266 - val_mae: 1.5632 - lr: 0.0082 - 2s/epoch - 2ms/step
Epoch 6/100
1000/1000 - 2s - loss: 16.

...

Score for fold 5: loss of 14.488956451416016
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100
1000/1000 - 20s - loss: 16.9003 - mse: 16.9003 - mae: 1.6519 - val_loss: 10.7619 - val_mse: 10.7619 - val_mae: 1.4665 - lr: 0.0026 - 20s/epoch - 20ms/step
Epoch 2/100
1000/1000 - 20s - loss: 16.5096 - mse: 16.5096 - mae: 1.6240 - val_loss: 10.6518 - val_mse: 10.6518 - val_mae: 1.6230 - lr: 0.0026 - 20s/epoch - 20ms/step
Epoch 3/100
1000/1000 - 19s - loss: 16.5830 - mse: 16.5830 - mae: 1.6206 - val_loss: 10.0838 - val_mse: 10.0838 - val_mae: 1.6448 - lr: 0.0026 - 19s/epoch - 19ms/step
Epoch 4/100
1000/1000 - 19s - loss: 16.5102 - mse: 16.5102 - mae: 1.6187 - val_loss: 10.1582 - val_mse: 10.1582 - val_mae: 1.6388 - lr: 0.0026 - 19s/epoch - 19ms/step
Epoch 5/100
1000/1000 - 19s - loss: 16.4208 - mse: 16.4208 - mae: 1.6173 - val_loss: 10.1047 - val_mse: 10.1047 - val_mae: 1.6698 - lr: 0.0026 - 19s/epoch - 19ms/step
Epoch 6/100
1000/1000 -

[I 2022-10-15 09:21:16,502] Finished trial#48 resulted in value: 14.626. Current best value is 14.552000000000001 with parameters: {'activation': 'relu', 'num_hidden_layer': 3, 'i': 9, 'learning_rate': 0.001648783292938752}.


Score for fold 5: loss of 15.96181583404541
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100
1000/1000 - 3s - loss: 16.1275 - mse: 16.1275 - mae: 1.6505 - val_loss: 14.8619 - val_mse: 14.8619 - val_mae: 1.6000 - lr: 0.0015 - 3s/epoch - 3ms/step
Epoch 2/100
1000/1000 - 2s - loss: 16.0216 - mse: 16.0216 - mae: 1.6289 - val_loss: 14.7681 - val_mse: 14.7681 - val_mae: 1.6193 - lr: 0.0015 - 2s/epoch - 2ms/step
Epoch 3/100
1000/1000 - 2s - loss: 15.9716 - mse: 15.9716 - mae: 1.6314 - val_loss: 14.7108 - val_mse: 14.7108 - val_mae: 1.6682 - lr: 0.0015 - 2s/epoch - 2ms/step
Epoch 4/100
1000/1000 - 2s - loss: 15.9531 - mse: 15.9531 - mae: 1.6339 - val_loss: 14.6141 - val_mse: 14.6141 - val_mae: 1.7305 - lr: 0.0015 - 2s/epoch - 2ms/step
Epoch 5/100
1000/1000 - 2s - loss: 15.9678 - mse: 15.9678 - mae: 1.6316 - val_loss: 14.7638 - val_mse: 14.7638 - val_mae: 1.6477 - lr: 0.0015 - 2s/epoch - 2ms/step
Epoch 6/100
1000/1000 - 2s - loss: 15.9

[I 2022-10-15 09:23:30,124] Finished trial#49 resulted in value: 15.824000000000002. Current best value is 14.552000000000001 with parameters: {'activation': 'relu', 'num_hidden_layer': 3, 'i': 9, 'learning_rate': 0.001648783292938752}.


Score for fold 5: loss of 14.840733528137207


In [ ]:
#{'activation': 'relu', 'num_hidden_layer': 3, 'i': 9, 'learning_rate': 0.001648783292938752}.
optimizer = Adam(learning_rate= 0.001648783292938752 ,clipnorm=1.0)
model_4 = create_model(activation="relu",num_hidden_layer=3,num_hidden_unit=512)

es = EarlyStopping(monitor='mse', patience=5)
reduce_lr = ReduceLROnPlateau(monitor='val_mse', factor=0.2,
                              patience=20, min_lr=0.001)
model_4.compile(loss='mse',optimizer=optimizer,metrics=['mse','mae'])
history = model_4.fit(training,labelsForTrain,
                batch_size=64,
                epochs=20,
                verbose=2,
                validation_data=(valing,labelsForVal),
                validation_batch_size=64,
                callbacks=[es,reduce_lr])

Epoch 1/20
1250/1250 - 4s - loss: 15.3966 - mse: 15.3966 - mae: 1.6204 - val_loss: 13.5829 - val_mse: 13.5829 - val_mae: 1.5739 - lr: 0.0016 - 4s/epoch - 3ms/step
Epoch 2/20
1250/1250 - 4s - loss: 15.2353 - mse: 15.2353 - mae: 1.6009 - val_loss: 13.7331 - val_mse: 13.7331 - val_mae: 1.7328 - lr: 0.0016 - 4s/epoch - 3ms/step
Epoch 3/20
1250/1250 - 4s - loss: 15.1292 - mse: 15.1292 - mae: 1.6030 - val_loss: 13.5205 - val_mse: 13.5205 - val_mae: 1.5912 - lr: 0.0016 - 4s/epoch - 3ms/step
Epoch 4/20
1250/1250 - 3s - loss: 15.1487 - mse: 15.1487 - mae: 1.6013 - val_loss: 13.6057 - val_mse: 13.6057 - val_mae: 1.5702 - lr: 0.0016 - 3s/epoch - 3ms/step
Epoch 5/20
1250/1250 - 3s - loss: 15.0921 - mse: 15.0921 - mae: 1.5955 - val_loss: 13.3360 - val_mse: 13.3360 - val_mae: 1.6124 - lr: 0.0016 - 3s/epoch - 3ms/step
Epoch 6/20
1250/1250 - 3s - loss: 15.0889 - mse: 15.0889 - mae: 1.5906 - val_loss: 13.6683 - val_mse: 13.6683 - val_mae: 1.6144 - lr: 0.0016 - 3s/epoch - 3ms/step
Epoch 7/20
1250/1250 -

In [ ]:
results_model4 = model_4.evaluate(testing, labelsForTest, batch_size=32)

313/313 [==============================] - 0s 1ms/step - loss: 11.7363 - mse: 11.7363 - mae: 1.5377


## Shuffle Repetation 5

In [ ]:
# Shuffle the data after train
shuffled4 = shuffle(train_df, random_state=4)
training_shuffled4,labelsForTrain_shuffled4=process_shuffle_dataset(shuffled4)

In [ ]:
model_list=[]
history_list=[]
study_name = 'NN_study_5'
study = optuna.create_study(study_name=study_name, load_if_exists=True)
func = lambda trial: objective(trial,training_shuffled4,labelsForTrain_shuffled4)
study.optimize(func, n_trials=50,)

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100
1000/1000 - 3s - loss: 18.1175 - mse: 18.1175 - mae: 1.7368 - val_loss: 15.2867 - val_mse: 15.2867 - val_mae: 1.6144 - lr: 1.9163e-04 - 3s/epoch - 3ms/step
Epoch 2/100
1000/1000 - 4s - loss: 15.9689 - mse: 15.9689 - mae: 1.6161 - val_loss: 15.0112 - val_mse: 15.0112 - val_mae: 1.6072 - lr: 1.9163e-04 - 4s/epoch - 4ms/step
Epoch 3/100
1000/1000 - 3s - loss: 15.7510 - mse: 15.7510 - mae: 1.6028 - val_loss: 14.8815 - val_mse: 14.8815 - val_mae: 1.6053 - lr: 1.9163e-04 - 3s/epoch - 3ms/step
Epoch 4/100
1000/1000 - 3s - loss: 15.6120 - mse: 15.6120 - mae: 1.6000 - val_loss: 14.8071 - val_mse: 14.8071 - val_mae: 1.5896 - lr: 1.9163e-04 - 3s/epoch - 3ms/step
Epoch 5/100
1000/1000 - 3s - loss: 15.5397 - mse: 15.5397 - mae: 1.5938 - val_loss: 14.7412 - val_mse: 14.7412 - val_mae: 1.5965 - lr: 1.9163e-04 - 3s/epoch - 3ms/step
Epoch 6/100
1000/1000 - 2s - loss: 15.4823 - mse: 15.4823 - mae

[I 2022-10-15 09:28:37,791] Finished trial#0 resulted in value: 14.958000000000002. Current best value is 14.958000000000002 with parameters: {'activation': 'relu', 'num_hidden_layer': 2, 'i': 6, 'learning_rate': 0.00019163230472226588}.


------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100
1000/1000 - 3s - loss: 15.9022 - mse: 15.9022 - mae: 1.6207 - val_loss: 13.8443 - val_mse: 13.8443 - val_mae: 1.5760 - lr: 6.3354e-04 - 3s/epoch - 3ms/step
Epoch 2/100
1000/1000 - 2s - loss: 15.4485 - mse: 15.4485 - mae: 1.5953 - val_loss: 14.1652 - val_mse: 14.1652 - val_mae: 1.5305 - lr: 6.3354e-04 - 2s/epoch - 2ms/step
Epoch 3/100
1000/1000 - 2s - loss: 15.4032 - mse: 15.4032 - mae: 1.5935 - val_loss: 13.8331 - val_mse: 13.8331 - val_mae: 1.6129 - lr: 6.3354e-04 - 2s/epoch - 2ms/step
Epoch 4/100
1000/1000 - 2s - loss: 15.3698 - mse: 15.3698 - mae: 1.5903 - val_loss: 13.7752 - val_mse: 13.7752 - val_mae: 1.5240 - lr: 6.3354e-04 - 2s/epoch - 2ms/step
Epoch 5/100
1000/1000 - 2s - loss: 15.3593 - mse: 15.3593 - mae: 1.5879 - val_loss: 13.6717 - val_mse: 13.6717 - val_mae: 1.5926 - lr: 6.3354e-04 - 2s/epoch - 2ms/step
Epoch 6/100
1000/1000 - 2s - loss: 15.3101 - mse: 15.3101 - mae

[I 2022-10-15 09:30:32,418] Finished trial#1 resulted in value: 14.922. Current best value is 14.922 with parameters: {'activation': 'relu', 'num_hidden_layer': 3, 'i': 7, 'learning_rate': 0.0006335354564972164}.


Score for fold 5: loss of 14.533296585083008
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100
1000/1000 - 37s - loss: 29.3071 - mse: 29.3071 - mae: 2.1981 - val_loss: 17.0357 - val_mse: 17.0357 - val_mae: 1.6009 - lr: 0.0060 - 37s/epoch - 37ms/step
Epoch 2/100
1000/1000 - 36s - loss: 17.9744 - mse: 17.9744 - mae: 1.9549 - val_loss: 16.2100 - val_mse: 16.2100 - val_mae: 1.6271 - lr: 0.0060 - 36s/epoch - 36ms/step
Epoch 3/100
1000/1000 - 37s - loss: 17.6447 - mse: 17.6447 - mae: 1.9134 - val_loss: 19.1415 - val_mse: 19.1415 - val_mae: 1.8652 - lr: 0.0060 - 37s/epoch - 37ms/step
Epoch 4/100
1000/1000 - 38s - loss: 17.6546 - mse: 17.6546 - mae: 1.9241 - val_loss: 20.1910 - val_mse: 20.1910 - val_mae: 2.1977 - lr: 0.0060 - 38s/epoch - 38ms/step
Epoch 5/100
1000/1000 - 39s - loss: 17.2029 - mse: 17.2029 - mae: 1.8998 - val_loss: 16.0543 - val_mse: 16.0543 - val_mae: 1.6408 - lr: 0.0060 - 39s/epoch - 39ms/step
Epoch 6/100
1000/1000 -

...

[I 2022-10-15 18:26:05,710] Finished trial#47 resulted in value: 14.556000000000001. Current best value is 14.538 with parameters: {'activation': 'relu', 'num_hidden_layer': 3, 'i': 10, 'learning_rate': 0.00010263970460536851}.


Score for fold 5: loss of 17.849748611450195
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100
1000/1000 - 6s - loss: 16.4922 - mse: 16.4922 - mae: 1.6235 - val_loss: 11.0034 - val_mse: 11.0034 - val_mae: 1.5637 - lr: 4.7069e-04 - 6s/epoch - 6ms/step
Epoch 2/100
1000/1000 - 7s - loss: 16.1381 - mse: 16.1381 - mae: 1.6068 - val_loss: 10.8518 - val_mse: 10.8518 - val_mae: 1.6322 - lr: 4.7069e-04 - 7s/epoch - 7ms/step
Epoch 3/100
1000/1000 - 5s - loss: 16.0696 - mse: 16.0696 - mae: 1.6055 - val_loss: 11.0517 - val_mse: 11.0517 - val_mae: 1.5327 - lr: 4.7069e-04 - 5s/epoch - 5ms/step
Epoch 4/100
1000/1000 - 5s - loss: 16.0402 - mse: 16.0402 - mae: 1.6025 - val_loss: 10.9720 - val_mse: 10.9720 - val_mae: 1.5187 - lr: 4.7069e-04 - 5s/epoch - 5ms/step
Epoch 5/100
1000/1000 - 5s - loss: 15.9781 - mse: 15.9781 - mae: 1.5898 - val_loss: 10.8487 - val_mse: 10.8487 - val_mae: 1.5927 - lr: 4.7069e-04 - 5s/epoch - 5ms/step
Epoch 6/100
1000/1

[I 2022-10-15 18:30:53,621] Finished trial#48 resulted in value: 14.778. Current best value is 14.538 with parameters: {'activation': 'relu', 'num_hidden_layer': 3, 'i': 10, 'learning_rate': 0.00010263970460536851}.


Score for fold 5: loss of 15.656357765197754
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100
1000/1000 - 3s - loss: 16.2725 - mse: 16.2725 - mae: 1.6726 - val_loss: 14.6213 - val_mse: 14.6213 - val_mae: 1.5838 - lr: 3.4584e-04 - 3s/epoch - 3ms/step
Epoch 2/100
1000/1000 - 3s - loss: 15.6579 - mse: 15.6579 - mae: 1.6170 - val_loss: 14.5461 - val_mse: 14.5461 - val_mae: 1.6006 - lr: 3.4584e-04 - 3s/epoch - 3ms/step
Epoch 3/100
1000/1000 - 3s - loss: 15.5254 - mse: 15.5254 - mae: 1.6067 - val_loss: 14.5628 - val_mse: 14.5628 - val_mae: 1.5554 - lr: 3.4584e-04 - 3s/epoch - 3ms/step
Epoch 4/100
1000/1000 - 3s - loss: 15.4338 - mse: 15.4338 - mae: 1.6017 - val_loss: 14.4584 - val_mse: 14.4584 - val_mae: 1.5837 - lr: 3.4584e-04 - 3s/epoch - 3ms/step
Epoch 5/100
1000/1000 - 3s - loss: 15.3492 - mse: 15.3492 - mae: 1.5937 - val_loss: 14.4173 - val_mse: 14.4173 - val_mae: 1.6170 - lr: 3.4584e-04 - 3s/epoch - 3ms/step
Epoch 6/100
1000/1

[I 2022-10-15 18:33:49,687] Finished trial#49 resulted in value: 14.820000000000002. Current best value is 14.538 with parameters: {'activation': 'relu', 'num_hidden_layer': 3, 'i': 10, 'learning_rate': 0.00010263970460536851}.


In [ ]:
#{'activation': 'relu', 'num_hidden_layer': 3, 'i': 10, 'learning_rate': 0.00010263970460536851}.
optimizer = Adam(learning_rate=0.00010263970460536851 ,clipnorm=1.0)
model_5 = create_model(activation="relu",num_hidden_layer=3,num_hidden_unit=1024)

es = EarlyStopping(monitor='mse', patience=5)
reduce_lr = ReduceLROnPlateau(monitor='val_mse', factor=0.2,
                              patience=20, min_lr=0.001)
model_5.compile(loss='mse',optimizer=optimizer,metrics=['mse','mae'])
history = model_5.fit(training,labelsForTrain,
                batch_size=64,
                epochs=20,
                verbose=2,
                validation_data=(valing,labelsForVal),
                validation_batch_size=64,
                callbacks=[es,reduce_lr])

Epoch 1/20
1250/1250 - 7s - loss: 15.5073 - mse: 15.5073 - mae: 1.6090 - val_loss: 13.5538 - val_mse: 13.5538 - val_mae: 1.6833 - lr: 1.0264e-04 - 7s/epoch - 5ms/step
Epoch 2/20
1250/1250 - 6s - loss: 15.0832 - mse: 15.0832 - mae: 1.5857 - val_loss: 13.5070 - val_mse: 13.5070 - val_mae: 1.6696 - lr: 1.0264e-04 - 6s/epoch - 5ms/step
Epoch 3/20
1250/1250 - 6s - loss: 15.0662 - mse: 15.0662 - mae: 1.5820 - val_loss: 13.6071 - val_mse: 13.6071 - val_mae: 1.5636 - lr: 1.0264e-04 - 6s/epoch - 5ms/step
Epoch 4/20
1250/1250 - 6s - loss: 15.0110 - mse: 15.0110 - mae: 1.5838 - val_loss: 13.3334 - val_mse: 13.3334 - val_mae: 1.6444 - lr: 1.0264e-04 - 6s/epoch - 5ms/step
Epoch 5/20
1250/1250 - 6s - loss: 14.9340 - mse: 14.9340 - mae: 1.5800 - val_loss: 13.5074 - val_mse: 13.5074 - val_mae: 1.5521 - lr: 1.0264e-04 - 6s/epoch - 5ms/step
Epoch 6/20
1250/1250 - 6s - loss: 14.9375 - mse: 14.9375 - mae: 1.5786 - val_loss: 13.3518 - val_mse: 13.3518 - val_mae: 1.6075 - lr: 1.0264e-04 - 6s/epoch - 5ms/ste

In [ ]:
results_model5 = model_5.evaluate(testing, labelsForTest, batch_size=32)

313/313 [==============================] - 1s 2ms/step - loss: 11.6360 - mse: 11.6360 - mae: 1.5715


##Shuffle Repetation 6

In [ ]:
# Shuffle the data after train
shuffled5 = shuffle(train_df, random_state=5)
training_shuffled5,labelsForTrain_shuffled5=process_shuffle_dataset(shuffled5)

In [ ]:
model_list=[]
history_list=[]
study_name = 'NN_study_6'
study = optuna.create_study(study_name=study_name, load_if_exists=True)
func = lambda trial: objective(trial,training_shuffled5,labelsForTrain_shuffled5)
study.optimize(func, n_trials=50,)

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100
1000/1000 - 3s - loss: 16.2545 - mse: 16.2545 - mae: 1.6608 - val_loss: 16.6368 - val_mse: 16.6368 - val_mae: 1.5995 - lr: 2.1064e-04 - 3s/epoch - 3ms/step
Epoch 2/100
1000/1000 - 2s - loss: 15.1146 - mse: 15.1146 - mae: 1.5999 - val_loss: 16.3394 - val_mse: 16.3394 - val_mae: 1.5698 - lr: 2.1064e-04 - 2s/epoch - 2ms/step
Epoch 3/100
1000/1000 - 2s - loss: 14.9362 - mse: 14.9362 - mae: 1.5920 - val_loss: 16.2105 - val_mse: 16.2105 - val_mae: 1.6078 - lr: 2.1064e-04 - 2s/epoch - 2ms/step
Epoch 4/100
1000/1000 - 2s - loss: 14.8638 - mse: 14.8638 - mae: 1.5949 - val_loss: 16.3082 - val_mse: 16.3082 - val_mae: 1.5623 - lr: 2.1064e-04 - 2s/epoch - 2ms/step
Epoch 5/100
1000/1000 - 2s - loss: 14.8344 - mse: 14.8344 - mae: 1.5914 - val_loss: 16.2764 - val_mse: 16.2764 - val_mae: 1.5568 - lr: 2.1064e-04 - 2s/epoch - 2ms/step
Epoch 6/100
1000/1000 - 2s - loss: 14.8199 - mse: 14.8199 - mae

[I 2022-10-16 05:45:07,257] Finished trial#0 resulted in value: 14.879999999999999. Current best value is 14.879999999999999 with parameters: {'activation': 'relu', 'num_hidden_layer': 3, 'i': 6, 'learning_rate': 0.00021063978488417066}.


Score for fold 5: loss of 15.603734016418457
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100
1000/1000 - 4s - loss: 16.7996 - mse: 16.7996 - mae: 1.6747 - val_loss: 11.8674 - val_mse: 11.8674 - val_mae: 1.6322 - lr: 6.6448e-04 - 4s/epoch - 4ms/step
Epoch 2/100
1000/1000 - 4s - loss: 16.4184 - mse: 16.4184 - mae: 1.6336 - val_loss: 12.2021 - val_mse: 12.2021 - val_mae: 1.5819 - lr: 6.6448e-04 - 4s/epoch - 4ms/step
Epoch 3/100
1000/1000 - 4s - loss: 16.3251 - mse: 16.3251 - mae: 1.6265 - val_loss: 11.8450 - val_mse: 11.8450 - val_mae: 1.6159 - lr: 6.6448e-04 - 4s/epoch - 4ms/step
Epoch 4/100
1000/1000 - 4s - loss: 16.2582 - mse: 16.2582 - mae: 1.6213 - val_loss: 11.6132 - val_mse: 11.6132 - val_mae: 1.5805 - lr: 6.6448e-04 - 4s/epoch - 4ms/step
Epoch 5/100
1000/1000 - 4s - loss: 16.1834 - mse: 16.1834 - mae: 1.6166 - val_loss: 11.8516 - val_mse: 11.8516 - val_mae: 1.5262 - lr: 6.6448e-04 - 4s/epoch - 4ms/step
Epoch 6/100
1000/1

[I 2022-10-16 05:48:07,734] Finished trial#1 resulted in value: 14.966. Current best value is 14.879999999999999 with parameters: {'activation': 'relu', 'num_hidden_layer': 3, 'i': 6, 'learning_rate': 0.00021063978488417066}.


Score for fold 5: loss of 18.452739715576172
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100
1000/1000 - 7s - loss: 16.1393 - mse: 16.1393 - mae: 1.8656 - val_loss: 22.1841 - val_mse: 22.1841 - val_mae: 1.8592 - lr: 0.0077 - 7s/epoch - 7ms/step
Epoch 2/100
1000/1000 - 6s - loss: 15.5685 - mse: 15.5685 - mae: 1.8162 - val_loss: 21.2417 - val_mse: 21.2417 - val_mae: 1.8389 - lr: 0.0077 - 6s/epoch - 6ms/step
Epoch 3/100
1000/1000 - 7s - loss: 15.3709 - mse: 15.3709 - mae: 1.7837 - val_loss: 21.7404 - val_mse: 21.7404 - val_mae: 1.7751 - lr: 0.0077 - 7s/epoch - 7ms/step
Epoch 4/100
1000/1000 - 6s - loss: 15.3673 - mse: 15.3673 - mae: 1.8026 - val_loss: 23.2597 - val_mse: 23.2597 - val_mae: 1.7769 - lr: 0.0077 - 6s/epoch - 6ms/step
Epoch 5/100
1000/1000 - 7s - loss: 15.4532 - mse: 15.4532 - mae: 1.8033 - val_loss: 22.0814 - val_mse: 22.0814 - val_mae: 1.8589 - lr: 0.0077 - 7s/epoch - 7ms/step
Epoch 6/100
1000/1000 - 6s - loss: 15.

...

Score for fold 5: loss of 26.396228790283203
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100
1000/1000 - 14s - loss: 15.6853 - mse: 15.6853 - mae: 1.6129 - val_loss: 14.3814 - val_mse: 14.3814 - val_mae: 1.6074 - lr: 1.1509e-04 - 14s/epoch - 14ms/step
Epoch 2/100
1000/1000 - 14s - loss: 15.2343 - mse: 15.2343 - mae: 1.5848 - val_loss: 14.5889 - val_mse: 14.5889 - val_mae: 1.5176 - lr: 1.1509e-04 - 14s/epoch - 14ms/step
Epoch 3/100
1000/1000 - 14s - loss: 15.2299 - mse: 15.2299 - mae: 1.5813 - val_loss: 14.4410 - val_mse: 14.4410 - val_mae: 1.6224 - lr: 1.1509e-04 - 14s/epoch - 14ms/step
Epoch 4/100
1000/1000 - 14s - loss: 15.1717 - mse: 15.1717 - mae: 1.5817 - val_loss: 14.6073 - val_mse: 14.6073 - val_mae: 1.5535 - lr: 1.1509e-04 - 14s/epoch - 14ms/step
Epoch 5/100
1000/1000 - 14s - loss: 15.0977 - mse: 15.0977 - mae: 1.5815 - val_loss: 14.3323 - val_mse: 14.3323 - val_mae: 1.6524 - lr: 1.1509e-04 - 14s/epoch - 14ms/step
Epo

[I 2022-10-16 09:28:27,809] Finished trial#48 resulted in value: 14.339999999999998. Current best value is 14.024000000000001 with parameters: {'activation': 'relu', 'num_hidden_layer': 4, 'i': 9, 'learning_rate': 0.00018064149922822524}.


Score for fold 5: loss of 16.35189437866211
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100
1000/1000 - 13s - loss: 15.6273 - mse: 15.6273 - mae: 1.6136 - val_loss: 14.3676 - val_mse: 14.3676 - val_mae: 1.5873 - lr: 1.1676e-04 - 13s/epoch - 13ms/step
Epoch 2/100
1000/1000 - 12s - loss: 15.2608 - mse: 15.2608 - mae: 1.5912 - val_loss: 14.5282 - val_mse: 14.5282 - val_mae: 1.5400 - lr: 1.1676e-04 - 12s/epoch - 12ms/step
Epoch 3/100
1000/1000 - 13s - loss: 15.2119 - mse: 15.2119 - mae: 1.5899 - val_loss: 14.5358 - val_mse: 14.5358 - val_mae: 1.5504 - lr: 1.1676e-04 - 13s/epoch - 13ms/step
Epoch 4/100
1000/1000 - 13s - loss: 15.1697 - mse: 15.1697 - mae: 1.5871 - val_loss: 14.7235 - val_mse: 14.7235 - val_mae: 1.5360 - lr: 1.1676e-04 - 13s/epoch - 13ms/step
Epoch 5/100
1000/1000 - 13s - loss: 15.0745 - mse: 15.0745 - mae: 1.5871 - val_loss: 14.4831 - val_mse: 14.4831 - val_mae: 1.5661 - lr: 1.1676e-04 - 13s/epoch - 13ms/step
Epoc

[I 2022-10-16 09:35:31,851] Finished trial#49 resulted in value: 14.706. Current best value is 14.024000000000001 with parameters: {'activation': 'relu', 'num_hidden_layer': 4, 'i': 9, 'learning_rate': 0.00018064149922822524}.


Score for fold 5: loss of 8.946612358093262


In [ ]:
#{'activation': 'relu', 'num_hidden_layer': 4, 'i': 9, 'learning_rate': 0.00018064149922822524}..
optimizer = Adam(learning_rate=0.00018064149922822524 ,clipnorm=1.0)
model_6 = create_model(activation="relu",num_hidden_layer=4,num_hidden_unit=512)

es = EarlyStopping(monitor='mse', patience=5)
reduce_lr = ReduceLROnPlateau(monitor='val_mse', factor=0.2,
                              patience=20, min_lr=0.001)
model_6.compile(loss='mse',optimizer=optimizer,metrics=['mse','mae'])
history = model_6.fit(training,labelsForTrain,
                batch_size=64,
                epochs=20,
                verbose=2,
                validation_data=(valing,labelsForVal),
                validation_batch_size=64,
                callbacks=[es,reduce_lr])

Epoch 1/20
1250/1250 - 6s - loss: 15.4598 - mse: 15.4598 - mae: 1.6063 - val_loss: 13.7991 - val_mse: 13.7991 - val_mae: 1.5361 - lr: 1.8064e-04 - 6s/epoch - 5ms/step
Epoch 2/20
1250/1250 - 5s - loss: 15.0903 - mse: 15.0903 - mae: 1.5900 - val_loss: 13.6928 - val_mse: 13.6928 - val_mae: 1.5615 - lr: 1.8064e-04 - 5s/epoch - 4ms/step
Epoch 3/20
1250/1250 - 5s - loss: 15.0270 - mse: 15.0270 - mae: 1.5898 - val_loss: 13.3485 - val_mse: 13.3485 - val_mae: 1.6743 - lr: 1.8064e-04 - 5s/epoch - 4ms/step
Epoch 4/20
1250/1250 - 5s - loss: 14.9620 - mse: 14.9620 - mae: 1.5861 - val_loss: 13.4234 - val_mse: 13.4234 - val_mae: 1.5668 - lr: 1.8064e-04 - 5s/epoch - 4ms/step
Epoch 5/20
1250/1250 - 5s - loss: 14.9829 - mse: 14.9829 - mae: 1.5834 - val_loss: 13.3538 - val_mse: 13.3538 - val_mae: 1.5883 - lr: 1.8064e-04 - 5s/epoch - 4ms/step
Epoch 6/20
1250/1250 - 5s - loss: 14.8728 - mse: 14.8728 - mae: 1.5766 - val_loss: 13.2445 - val_mse: 13.2445 - val_mae: 1.6180 - lr: 1.8064e-04 - 5s/epoch - 4ms/ste

In [ ]:
results_model6 = model_6.evaluate(testing, labelsForTest, batch_size=32)

313/313 [==============================] - 1s 2ms/step - loss: 11.9252 - mse: 11.9252 - mae: 1.6769


The models performance on validation datasets:

In [ ]:
MSE_val=np.array([13.34,13.15,13.75,13.05,13.22,13.27])
RMSE_val=np.sqrt(MSE_val)
MAE_val=np.array([1.704,1.593,1.555,1.61,1.569,1.589])

In [ ]:
mean_RMSE_val=np.mean(RMSE_val)
mean_MAE_val=np.mean(MAE_val)

In [ ]:
print(mean_RMSE_val)
print(mean_MAE_val)

3.646333312697695
1.6033333333333333


In [ ]:
print(np.std(RMSE_val))
print(np.std(MAE_val))

0.030332150891770937
0.048313789151982506


## Combine the trainning dataset and validation dataset for training "the best of best" model

In [ ]:
train_val=pd.concat([train_df, val_df])

In [ ]:
train_val_input,labelsFortrain_val=process_shuffle_dataset(train_val)

In [ ]:
train_val_input

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,0.902011,0.995920,-0.804756,-0.330617,-0.379517,-0.206158,-0.188756,-0.610631,-0.172660,1.410127,...,1.010500,0.728954,1.191707,-0.797708,-0.915846,-0.878536,-0.600577,-0.037368,-0.012793,-0.565006
1,0.902011,0.995920,-0.603161,-0.026909,0.119132,-1.204293,-0.111687,1.015724,0.012508,0.583623,...,0.362933,-1.006542,0.347735,-0.567887,-0.699348,-0.878536,-0.070421,-0.037368,-0.012793,0.222404
2,0.902011,0.995920,0.930712,0.589285,0.861888,2.393625,0.795618,3.292620,0.218251,0.535006,...,-1.024711,-0.059908,1.191707,1.270684,0.491392,1.120411,0.636455,-0.037368,-0.012793,-0.171301
3,0.253331,0.061782,-0.033437,-0.104152,-0.014398,2.512132,0.505354,1.015724,0.005650,-0.084872,...,-1.487259,0.728954,-0.496237,0.305434,0.058396,-0.434325,-0.423858,-0.037368,-0.012793,-0.565006
4,-1.692709,-1.495115,3.542680,0.059113,-0.394122,-1.233808,0.157354,-0.610631,-0.172660,-3.305804,...,-2.689883,0.886726,1.191707,3.109255,1.249135,3.563567,3.817395,-0.037368,-0.012793,-0.565006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89995,-0.395349,-0.664770,-0.883641,-0.330617,-0.358653,-0.920665,-0.188756,-0.610631,-0.165802,-0.024100,...,0.640461,-0.533225,-0.496237,-0.935601,-0.591099,-0.767483,-0.600577,-0.037368,-0.012793,-0.565006
89996,0.902011,0.995920,0.781707,0.194289,-0.068644,-0.004835,-0.056484,0.039911,-0.124653,0.413461,...,-0.562163,0.571181,1.191707,0.489291,0.707890,1.342516,0.283017,-0.037368,-0.012793,0.222404
89997,0.253331,0.061782,0.483698,0.066135,1.788247,0.171983,-0.123963,0.690453,0.122238,0.401307,...,-0.284634,-0.217680,0.347735,1.086827,0.924388,0.343043,-0.423858,-0.037368,-0.012793,-0.171301
89998,0.253331,0.061782,-0.243797,0.417242,1.992714,2.826385,-0.065426,3.292620,0.499433,0.510697,...,0.270423,-0.533225,-0.496237,-0.154209,0.383143,0.343043,-0.070421,-0.037368,-0.012793,-0.565006


In [ ]:
#{'activation': 'relu', 'num_hidden_layer': 4, 'i': 10, 'learning_rate': 0.00015658492778156685}.
optimizer = Adam(learning_rate=0.00015658492778156685 ,clipnorm=1.0)
best_model = create_model(activation="relu",num_hidden_layer=4,num_hidden_unit=1024)

es = EarlyStopping(monitor='mse', patience=5)
reduce_lr = ReduceLROnPlateau(monitor='mse', factor=0.2,
                              patience=20, min_lr=0.001)
best_model.compile(loss='mse',optimizer=optimizer,metrics=['mse','mae'])
history = best_model.fit(train_val_input,labelsFortrain_val,
                batch_size=64,
                epochs=20,
                verbose=2,
                callbacks=[es,reduce_lr])

Epoch 1/20
1407/1407 - 13s - loss: 12.7619 - mse: 12.7619 - mae: 1.5371 - lr: 1.5658e-04 - 13s/epoch - 9ms/step
Epoch 2/20
1407/1407 - 12s - loss: 11.9341 - mse: 11.9341 - mae: 1.4849 - lr: 1.5658e-04 - 12s/epoch - 8ms/step
Epoch 3/20
1407/1407 - 12s - loss: 11.7715 - mse: 11.7715 - mae: 1.4672 - lr: 1.5658e-04 - 12s/epoch - 8ms/step
Epoch 4/20
1407/1407 - 12s - loss: 11.8170 - mse: 11.8170 - mae: 1.4617 - lr: 1.5658e-04 - 12s/epoch - 8ms/step
Epoch 5/20
1407/1407 - 12s - loss: 11.5307 - mse: 11.5307 - mae: 1.4523 - lr: 1.5658e-04 - 12s/epoch - 8ms/step
Epoch 6/20
1407/1407 - 12s - loss: 11.3214 - mse: 11.3214 - mae: 1.4419 - lr: 1.5658e-04 - 12s/epoch - 8ms/step
Epoch 7/20
1407/1407 - 12s - loss: 11.2257 - mse: 11.2257 - mae: 1.4355 - lr: 1.5658e-04 - 12s/epoch - 8ms/step
Epoch 8/20
1407/1407 - 12s - loss: 11.2936 - mse: 11.2936 - mae: 1.4294 - lr: 1.5658e-04 - 12s/epoch - 8ms/step
Epoch 9/20
1407/1407 - 12s - loss: 10.7149 - mse: 10.7149 - mae: 1.4193 - lr: 1.5658e-04 - 12s/epoch - 8

In [ ]:
results_best_model = best_model.evaluate(testing, labelsForTest, batch_size=32)

313/313 [==============================] - 1s 3ms/step - loss: 14.0053 - mse: 14.0053 - mae: 1.5175
